# PCA Python vs R

Originally, R was used to calculate PCA using both `princomp` and `prcomp`. However, rpy2 stopped was intorducing some issues on the galaxy server. I decided to switch the calculation over to a pure python solution. scikit-learn has a PCA package which we can used, but it only does SVD and matches the output of `prcomp` with its default values. 

Here I am testing and figuing out how to output the different values.

In [147]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

## Import Example Data

In [210]:
dat = pd.read_table('../example_data/ST000015_log.tsv')
dat.set_index('Name', inplace=True)

In [211]:
dat[:3]

,a332577,a332581,a332585,a332589,a332593,a332597,a332601,a332469,a332477,a332481,...,a332337,a332341,a332345,a332349,a332217,a332221,a332225,a332229,a332233,a332237
Name,,,,,,,,,,,,,,,,,,,,,
tyrosine,7.3923,7.4998,7.1898,9.0168,6.3038,6.5236,8.2288,7.5999,6.7549,8.9858,...,14.8625,7.5774,9.0688,14.4720,14.1080,11.2697,8.6366,11.4538,10.9766,14.1164
tryptophan,6.4594,7.1189,5.2095,7.0444,5.9542,7.3309,8.0056,4.0000,5.2095,3.0000,...,8.5507,7.2192,7.5314,8.0389,6.4757,9.2021,9.2503,8.5962,8.8202,10.9665
trehalose,8.9571,8.8361,9.5699,8.9129,8.0875,8.8234,9.0389,9.5565,8.8549,9.8074,...,10.2131,9.3061,9.1344,11.7385,11.0841,10.4949,11.3663,10.4737,10.6165,11.4949


## Use R to calculate PCA

Mi has looked at this already, but wanted to put the R example here to be complete. Here are the two R methods to output PCA

In [221]:
%%R -i dat
# First method uses princomp to calulate PCA using eigenvalues and eigenvectors
pr = princomp(dat)
#str(pr)
loadings = pr$loadings
scores = pr$scores
#summary(pr)

[1] 12.61808


In [223]:
%%R -i dat
pr = prcomp(dat)
#str(pr)
loadings = pr$rotation
scores = pr$x
sd = pr$sdev
#summary(pr)

## Use Python to calculate PCA

scikit-learn has a PCA package that we will use. It uses the SVD method, so results match the prcomp from R. 

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


### Generate PCA with default settings

In [181]:
# Initiate PCA class
pca = PCA()

# Fit the model and transform data
scores = pca.fit_transform(dat)

# Get loadings
loadings = pca.components_

# R also outputs the following in their summaries
sd = loadings.std(axis=0)
propVar = pca.explained_variance_ratio_
cumPropVar = propVar.cumsum()

I compared these results with prcomp and they are identical, note that the python version formats the data in scientific notation.

### Build output tables that match the original PCA script

#### Build comment block

At the top of each output file, the original R version includes the standard deviation and the proportion of variance explained. I want to first build this block.

In [148]:
# Labels used for the comment block
labels = np.array(['#Std. deviation', '#Proportion of variance explained', '#Cumulative proportion of variance explained'])

# Stack the data into a matrix
data = np.vstack([sd, propVar, cumPropVar])

# Add the labels to the first position in the matrix
block = np.column_stack([labels, data])

In [189]:
# Create header
header = np.array(['Comp{}'.format(x+1) for x in range(loadings.shape[1])])
compoundIndex = np.hstack([dat.index.name, dat.index])
sampleIndex = np.hstack(['sampleID', dat.columns])

# Create loadings output
loadHead = np.vstack([header, loadings])
loadIndex = np.column_stack([sampleIndex, loadHead])
loadOut = np.vstack([block, loadIndex])

# Create scores output
scoreHead = np.vstack([header, scores])
scoreIndex = np.column_stack([compoundIndex, scoreHead])
scoreOut = np.vstack([block, scoreIndex])

In [198]:
np.savetxt('/home/jfear/tmp/dan.tsv', loadOut, fmt="%s", delimiter='\t')

In [199]:
bob = pd.DataFrame(loadOut)

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,#Std. deviation,0.0894106208128,0.0872859231919,0.0889353873848,0.0883950492616,0.0894074630293,0.089269416936,0.0889222157093,0.0894313538218,0.0878101808881,...,0.0894095300328,0.0889152037066,0.0889979196178,0.0893452895665,0.0893128108836,0.0871128038213,0.0893214244514,0.089437875108,0.0887774951295,0.0893028422611
1,#Proportion of variance explained,0.768408398347,0.0944852586284,0.0166584048729,0.0106266237539,0.00979897186803,0.00790398627099,0.00732960958491,0.00588902805794,0.00556921465521,...,2.03871235916e-05,1.77822280914e-05,1.60299095197e-05,1.57034683108e-05,1.4590216401e-05,1.39194117166e-05,1.10429619828e-05,1.0236398879e-05,9.40237128738e-06,7.5601070231e-06
2,#Cumulative proportion of variance explained,0.768408398347,0.862893656976,0.879552061849,0.890178685603,0.899977657471,0.907881643742,0.915211253327,0.921100281384,0.92666949604,...,0.999883732927,0.999901515155,0.999917545064,0.999933248533,0.999947838749,0.999961758161,0.999972801123,0.999983037522,0.999992439893,1.0
3,sampleID,Comp1,Comp2,Comp3,Comp4,Comp5,Comp6,Comp7,Comp8,Comp9,...,Comp116,Comp117,Comp118,Comp119,Comp120,Comp121,Comp122,Comp123,Comp124,Comp125
4,a332577,0.097521808139,0.0937277288788,0.0900266523601,0.0965259218793,0.0945804888365,0.0970243534637,0.0935588763484,0.0993381789884,0.0965569665538,...,0.0851522283248,0.0892410068816,0.0925691207242,0.090868813658,0.0843003172034,0.089551280584,0.087897557946,0.0867912987454,0.0901725818397,0.0767976229222
5,a332581,0.0925972471236,0.0769180431411,0.0892523921678,0.0873247382167,0.0840772828338,0.0982211833779,0.0897438328396,0.0988180185308,0.0690570535319,...,-0.105824086404,0.0799387123105,0.0793653097114,-0.0770423892572,-0.0815753908256,-0.0691608198608,-0.0261685156676,-0.0834444671331,-0.0836050600954,-0.0809092929226
6,a332585,-0.0842646917484,-0.0163160762014,0.0727580492313,0.0153574625405,0.0473210526089,-0.00198109684667,0.027559067809,0.0623598894243,0.0705554691894,...,-0.123524052511,0.104494306589,0.0204372188111,-0.182617444344,-0.0357474982882,-0.0753856867737,-0.182450752028,0.0263266986834,0.0523622199735,-0.133972666153
7,a332589,0.125953213918,-0.000500428164759,-0.0324524891979,-0.000166037762408,-0.020308727673,0.0366006201004,-0.00609119044521,0.164334493569,0.0735971161439,...,0.00405438104658,-0.00699084676547,-0.14202858728,0.0843406796135,-0.0370635176669,0.129212713823,0.108886295388,0.140488207108,0.0127816140753,0.086256573452
8,a332593,-0.00987884443542,-0.0458308002471,-0.0177607968336,-0.00232401832146,0.00181068726625,0.0610546639988,0.003475539995,0.0350825436376,-0.0488697491487,...,-0.000911134629079,-0.13622522337,-0.19400500508,-0.0174597896632,-0.115094528371,0.128857213084,0.0918907312627,0.0585870813712,0.0542738234461,0.0472667540788
9,a332597,0.0975572099088,0.0516611302958,0.00808207391741,-0.0526941567382,-0.043137639902,-0.141360265669,-0.0772084746394,0.00794485000027,0.0220565512128,...,0.115730339818,-0.122469254812,-0.0741445842918,-0.144057328623,-0.1922262536,-0.0559079384615,0.0425062596227,-0.031019657583,-0.148447152661,-0.151211451821
